In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('Db-IMDB.db')
table_name = pd.read_sql_query('''SELECT name FROM sqlite_master WHERE type ='table' AND name NOT LIKE 'sqlite_%';''',conn)
table_name

,name
0,Movie
1,Genre
2,Language
3,Country
4,Location
5,M_Location
6,M_Country
7,M_Language
8,M_Genre
9,Person


Q1: List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

In [2]:
Query_1 = pd.read_sql_query('''SELECT p.Name,m.title,SUBSTR(m.year,-4) as yr,g.Name FROM Person p 
                               join M_Director md  on p.PID =  md.PID 
                               join Movie m on m.MID = md.MID   
                               join M_Genre mg on mg.MID = m.MID 
                               join Genre g on g.GID = mg.GID
                               WHERE g.Name like '%Comedy%' AND ((yr % 4 = 0 AND yr %100 != 0) OR (yr % 400 = 0) ) 
                               ORDER BY yr''',conn)
Query_1.head()

,Name,title,yr,Name
0,Amit Mitra,Jagte Raho,1956,"Comedy, Drama"
1,Chetan Anand,Funtoosh,1956,"Comedy, Drama"
2,Chetan Anand,Funtoosh,1956,"Comedy, Drama"
3,Satyen Bose,Jagriti,1956,"Comedy, Drama, Family"
4,Mohan Segal,New Delhi,1956,"Comedy, Drama, Musical"


In [3]:
Query_1.shape

(388, 4)

Q2: List the names of all the actors who played in the movie 'Anand' (1971)

In [4]:
Query_2 = pd.read_sql_query('''SELECT p.Name,m.title FROM Person p 
                               JOIN M_Cast mc on TRIM(mc.PID) = p.PID
                               JOIN Movie m ON m.MID = mc.MID
                               WHERE m.title LIKE 'Anand'
                               ''',conn)
Query_2.head()

,Name,title
0,Amitabh Bachchan,Anand
1,Rajesh Khanna,Anand
2,Brahm Bhardwaj,Anand
3,Ramesh Deo,Anand
4,Seema Deo,Anand


In [5]:
Query_2.shape

(17, 2)

Q3: List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970
and > 1990.)

In [6]:
Query_3 = pd.read_sql_query('''SELECT  DISTINCT TRIM(p.Name)as Director_Name,p.PID FROM Person p 
                               JOIN M_Cast mc ON TRIM(mc.PID) = p.PID
                               JOIN Movie m ON m.MID = mc.MID
                               WHERE CAST(SUBSTR(m.year,-4) AS INTEGER) < 1970
                               INTERSECT
                               SELECT  DISTINCT TRIM(p.Name),p.PID FROM Person p 
                               JOIN M_Cast mc ON TRIM(mc.PID) = p.PID
                               JOIN Movie m ON m.MID = mc.MID
                               WHERE CAST(SUBSTR(m.year,-4) AS INTEGER) > 1990
                            
                                '''
                                ,conn)
Query_3.head()

,Director_Name,PID
0,A.K. Hangal,nm0359845
1,Aachi Manorama,nm7390393
2,Abbas,nm0007807
3,Abdul,nm0008248
4,Abhi Bhattacharya,nm0080337


In [7]:
Query_3.shape

(300, 2)

Q4: List all directors who directed 10 movies or more, in descending order of the number of
movies they directed. Return the directors' names and the number of movies each of them
directed

In [8]:
Query_4 = pd.read_sql_query('''SELECT DISTINCT p.name as Director_Name,COUNT(md.MID) FROM Person p 
                               JOIN M_Director md on md.PID = p.PID
                               GROUP BY Director_Name 
                               HAVING COUNT(md.MID) >= 10
                               ORDER BY COUNT(md.MID) DESC
                               ''',conn)
Query_4.head()

,Director_Name,COUNT(md.MID)
0,David Dhawan,78
1,Mahesh Bhatt,72
2,Ram Gopal Varma,60
3,Vikram Bhatt,58
4,Hrishikesh Mukherjee,54


In [9]:
Query_4.shape

(156, 2)

Q5a: For each year, count the number of movies in that year that had only female actors.

In [10]:
Query_5a = pd.read_sql_query(''' SELECT M.year,COUNT(M.MID) as num_female_movie FROM Movie M 
                                 WHERE NOT EXISTS (SELECT M.MID FROM Movie
                                 JOIN M_Cast C on C.MID = M.MID 
                                 JOIN Person P on P.PID = C.PID
                                 WHERE P.gender = 'Male')
                                 GROUP BY M.year''',conn)
Query_5a

,year,num_female_movie
0,1939,1
1,1999,1
2,2000,1
3,2009,1
4,2012,1
5,2018,2


Q5b: Now include a small change: report for each year the percentage of movies in that 
year with only female actors, and the total number of movies made that year. For
example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were
13,522 movies, and 31.81% had only female actors. You do not need to round youanswer.

In [11]:
Query_5b = pd.read_sql_query(''' select SUB_1.year, SUB_2.C as total_Movie,SUB_1.c*100.00/SUB_2.C as percentage 
                                 from (select M.year, count(*) as C
                                       from movie M where not exists
                                      (select * from Person P, M_Cast C1
                                      where P.PID = C1.PID and 
                                      C1.mid = M.MID and  NOT P.gender='Female')
                                      group by M.year) SUB_1,
                                      (select M2.year, count(*) as C from movie M2 group by M2.year) SUB_2
                                       where SUB_1.year=SUB_2.year
                                       order by SUB_1.year
                                 ''',conn)
Query_5b

,year,total_Movie,percentage
0,1939,2,50.000000
1,1999,66,1.515152
2,2000,64,1.562500
3,2009,110,0.909091
4,2012,111,0.900901
5,2018,104,1.923077


Q6: Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast
size" we mean the number of distinct actors that played in that movie: if an actor played
multiple roles, or if it simply occurs multiple times in casts, we still count her/him only
once.

In [12]:
Query_6 = pd.read_sql_query('''SELECT title,MAX(Cast_size) as Cast,MID FROM
                               (SELECT COUNT(PID) Cast_size,M.MID,M.title  FROM Movie M
                               JOIN M_Cast C on C.MID = M.MID 
                               GROUP BY M.MID)''',conn)
Query_6

,title,Cast,MID
0,Ocean's Eight,238,tt5164214


Q7: A decade is a sequence of 10 consecutive years. For example, say in your database you have
movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the
second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of
films and the total number of films in D

In [13]:
Query_7 = pd.read_sql_query(''' SELECT dec_start,dec_end,Max(num_movies) FROM (SELECT 
                               (year-yr.digit)/ 10 * 10 + yr.digit as dec_start, 
                               (year-yr.digit)/ 10 * 10 + yr.digit + 9 as dec_end,
                              COUNT (TRIM(MID)) as num_movies
                              FROM  Movie Cross JOIN (SELECT MIN(year) % 10 digit FROM Movie) yr
                              GROUP BY dec_start, dec_end 
                              ORDER BY year)''',conn)
Query_7

,dec_start,dec_end,Max(num_movies)
0,2001,2010,1047


OR If for each year it is considered as decade start

In [14]:
query_7 = pd.read_sql_query('''SELECT yr.year as dec_start,yr.year + 9 as dec_end,COUNT(DISTINCT m.MID) as num_movies
                             FROM  (SELECT DISTINCT year FROM Movie) yr ,Movie m WHERE m.year >= yr.year
                             AND m.year < yr.year + 10
                             GROUP BY yr.year 
                             ORDER BY COUNT(DISTINCT m.MID) DESC
                             ''',conn)
query_7.head()

,dec_start,dec_end,num_movies
0,2008,2017,1205
1,2009,2018,1202
2,2007,2016,1188
3,2005,2014,1170
4,2006,2015,1160


Q8: Find the actors that were never unemployed for more than 3 years at a stretch. (Assume
that the actors remain unemployed between two consecutive movies)

In [15]:
Query_8 = pd.read_sql_query('''Select DISTINCT B.*, (B.nxt_yr - B.year) as Gap From 
                               (Select A.PID,A.Name,A.year,
                                Lead(M.year,1,0) Over (Partition By A.Name Order By A.year ASC) as nxt_yr
                                From( (Select Distinct PID,Name from Person) P
                                     Join (Select Distinct MID,PID From M_Cast )C on P.PID = C.PID 
                                     Join (Select Distinct MID,title,year From Movie)M on M.MID = C.MID) A) B
                                Where B.nxt_yr > B.year AND (B.nxt_yr - B.year) < 3
                                Order By B.year
                                 ''',conn)
Query_8.head()

,PID,Name,year,nxt_yr,Gap
0,nm0405340,Olaf Hytten,1939,1941,2
1,nm0442470,Kamini Kaushal,1946,1948,2
2,nm0474801,Dilip Kumar,1947,1948,1
3,nm0534870,Madhubala,1947,1949,2
4,nm0420451,Noor Jehan,1947,1948,1


In [16]:
Query_8.shape

(21480, 5)

Q9: Find all the actors that made more movies with Yash Chopra than any other director

In [4]:
Query_9 = pd.read_sql_query('''SELECT Distinct P1.Name,P1.PID,Count(Distinct M1.MID) as Num_Movie_wid_YC FROM Person P1 
                         Join M_Cast C on C.PID = P1.PID
                         Join Movie M1 on M1.MID = C.MID
                         Join M_Director D on D.MID = M1.MID
                         Join Person P2 on D.PID = P2.PID 
                         Where P2.Name = 'Yash Chopra'
                         Group By P1.PID
                         Having Num_Movie_wid_YC >
                         (Select  Count (Distinct M2.MID) From Movie M2
                         Join M_Cast C2 on C2.MID = M2.MID
                         Join Person P3 on C2.PID = P3.PID
                         Join M_Director D2 on D2.MID = M2.MID
                         Join Person P4 on D2.PID = P4.PID
                         Where P3.PID = P1.PID and NOT P4.Name = 'Yash Chopra' 
                         Group By P3.PID,D2.PID)
                         Order By Num_Movie_wid_YC DESC
                         ''',conn)
Query_9.head()

,Name,PID,Num_Movie_wid_YC
0,Jagdish Raj,nm0707271,11
1,Manmohan Krishna,nm0471443,10
2,Iftekhar,nm0407002,9
3,Madan Puri,nm0534501,8
4,Vikas Anand,nm0025630,8


In [27]:
Query_9.shape

(65, 3)

Q10: The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2. 

In [46]:
Query_10 = pd.read_sql_query('''Select DISTINCT P3.Name From Person P3
                                Join M_Cast C4 on C4.PID = P3.PID
                                Join Movie M4 on M4.MID = C4.MID
                                Where M4.MID in (
                                Select DISTINCT M3.MID From Movie M3
                                Join M_Cast C3 on C3.MID = M3.MID
                                Where C3.PID in( 
                                Select Distinct P2.PID From Person P2
                                Join M_Cast C2 on C2.PID = P2.PID
                                Join Movie M2 on M2.MID = C2.MID
                                Where M2.MID IN (
                                Select Distinct M1.MID as SRK_Muvi_id From Person P1
                                Join M_Cast C1 on C1.PID = P1.PID
                                Join Movie M1 on M1.MID  = C1.MID  
                                Where P1.Name = 'Shah Rukh Khan'))
                                And Not M3.MID in (
                                Select Distinct M1.MID as SRK_Muvi_id From Person P1
                                Join M_Cast C1 on C1.PID = P1.PID
                                Join Movie M1 on M1.MID  = C1.MID  
                                Where P1.Name = 'Shah Rukh Khan'))
                                
                                 ''',conn)
Query_10.head()

,Name
0,Kristin Scott Thomas
1,Derek Jacobi
2,Nick Frost
3,Walton Goggins
4,Michael Obiora


In [47]:
Query_10.shape

(25612, 1)

Note:- Due to duplicate values count may differ than the actual results